In [1]:
from operator import itemgetter

import pandas as pd
from more_itertools import flatten
from sklearn.utils import gen_batches
from tqdm.notebook import tqdm

from nya_ml.models.blanchefort_rubert_sentiment import BlanchefortRuBertSentiment
from nya_ml.models.tatyana_rubert_sentiment import TatyanaRuBertSentiment
from nya_ml_research.config import DATA_PATH

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
blanchefort = BlanchefortRuBertSentiment.load()

In [4]:
tatyana = TatyanaRuBertSentiment.load()


KeyboardInterrupt



In [8]:
blanchefort.predict(['Привет всем любителям пончиков'])

{'neutral': 0.9178354144096375,
 'positive': 0.08072204142808914,
 'negative': 0.0014424966648221016}

In [12]:
tatyana.predict(['Привет всем любителям пончиков'])

{'NEUTRAL': 0.18066978454589844,
 'POSITIVE': 0.06783491373062134,
 'NEGATIVE': 0.7514952421188354}

In [5]:
rusentiment = pd.read_csv(DATA_PATH / 'raw' / 'sentiment' / 'rusentiment_random_posts.csv')

In [6]:
rusentiment.shape

(21268, 2)

In [32]:
rusentiment.head(10)

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...
5,neutral,Решите уравнение мужских дел для жизни :\nДомо...
6,positive,патамушта я крутая баба! ееее!!
7,neutral,Просто пост :)
8,positive,урря!я дождался этой овцыы)
9,speech,"С Днем Рождения желаю много счастья, любви и у..."


In [33]:
rusentiment.label.value_counts()

neutral     8323
positive    4635
skip        3190
speech      2826
negative    2294
Name: label, dtype: int64

In [34]:
rusentiment = rusentiment[rusentiment.label != 'speech']
rusentiment = rusentiment[rusentiment.label != 'skip']

In [35]:
rusentiment.label.value_counts()

neutral     8323
positive    4635
negative    2294
Name: label, dtype: int64

In [7]:
rutweet = pd.read_csv(DATA_PATH / 'raw' / 'sentiment' / 'ru-tweet-corp.csv', names=['text', 'label'], usecols=[4, 5])

In [8]:
rutweet.shape

(226834, 2)

In [39]:
rutweet.head(10)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
5,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1
6,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1
7,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1
8,Поприветствуем моего нового читателя @Alexey17...,1
9,Теперь у меня есть частичка Сиднея :) #Sydney ...,1


In [40]:
rutweet.label.value_counts()

 1    114911
-1    111923
Name: label, dtype: int64

In [ ]:
rutweet['label'].rename({-1: 2, 1: 1}, inplace=True)
rusentiment['label'].rename({'negative': 2, 'positive': 1, 'neutral': 0}, inplace=True)

In [45]:
predictions = []

In [ ]:
dataset = rutweet
model = blanchefort
# for dataset in [rusentiment, rutweet]:
for batch in tqdm(gen_batches(len(dataset), 64), total=int(len(dataset) / 64)):
    predictions.append(blanchefort.predict(dataset.text[batch].tolist()))


  0%|          | 0/3544 [00:00<?, ?it/s]

In [ ]:
y_pred = list(map(
    lambda pr: list(model._grad.keys()).index(max(pr.items(), key=itemgetter(1))[0]),
    flatten(predictions)
))